# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../WeatherPy/output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Adamstown,-25.0660,-130.1015,24.36,84,100,5.04,PN,1705909344
1,1,Papatowai,-46.5619,169.4708,15.37,94,94,1.74,NZ,1705909345
2,2,Mragowo,53.8644,21.3051,0.00,80,100,8.31,PL,1705909347
3,3,Port-Aux-Francais,-49.3500,70.2167,6.12,80,91,11.98,TF,1705909348
4,4,Pampierstad,-27.7832,24.6877,30.28,25,1,1.35,ZA,1705909350


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1400,
    frame_height = 1000,
    size = "Humidity",
    color = "Wind Speed",
    scale = 0.6,
   
)

# Display the map
display(map_plot_1)

c:\Users\bosb0\anaconda3.1\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Wind Speed,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= 17) &
                                  (city_data_df["Max Temp"] <= 28) &
                                  (city_data_df["Wind Speed"] >= 1.5) &
                                  (city_data_df["Cloudiness"] >= 20) &
                                  (city_data_df["Cloudiness"] <= 80) &
                                  (city_data_df["Humidity"] <= 60)]

# Drop any rows with null values
narrow_city_df.dropna()

# Display sample data
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
125,125,Kinmparana,12.8402,-4.9199,22.82,10,44,3.02,ML,1705909536
128,128,Bakhtiyarpur,25.4667,85.5167,18.01,31,35,2.90,IN,1705909540
170,170,Mangrol,21.1167,70.1167,26.30,35,48,4.05,IN,1705909603
279,279,Bandarbeyla,9.4942,50.8122,26.57,60,46,10.74,SO,1705909771
316,316,Gao,16.6362,1.6370,22.21,8,22,4.70,ML,1705909828
360,360,Hobyo,5.3505,48.5268,27.92,58,56,12.15,SO,1705909897
426,426,Butembo,0.1500,29.2833,25.43,24,72,1.94,CD,1705909997
436,436,Puerto Gaitan,4.3133,-72.0816,26.85,47,78,4.34,CO,1705910013
500,500,Mirpur Sakro,24.5474,67.6277,26.43,40,50,5.67,PK,1705910110


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df["Hotel Name"] = ""

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
hotel_df

C:\Users\bosb0\AppData\Local\Temp\ipykernel_30556\1972183331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
125,Kinmparana,ML,12.8402,-4.9199,10,
128,Bakhtiyarpur,IN,25.4667,85.5167,31,
170,Mangrol,IN,21.1167,70.1167,35,
279,Bandarbeyla,SO,9.4942,50.8122,60,
316,Gao,ML,16.6362,1.6370,8,
360,Hobyo,SO,5.3505,48.5268,58,
426,Butembo,CD,0.1500,29.2833,24,
436,Puerto Gaitan,CO,4.3133,-72.0816,47,
500,Mirpur Sakro,PK,24.5474,67.6277,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel

radius = 10000

limit = 1
params = {
    "filter":"",
    "limit":limit,
    "bias": "",
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Kinmparana - nearest hotel: No hotel found
Bakhtiyarpur - nearest hotel: No hotel found
Mangrol - nearest hotel: No hotel found
Bandarbeyla - nearest hotel: No hotel found
Gao - nearest hotel: No hotel found
Hobyo - nearest hotel: Iftin Hotel Hobyo
Butembo - nearest hotel: Canada House
Puerto Gaitan - nearest hotel: No hotel found
Mirpur Sakro - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
125,Kinmparana,ML,12.8402,-4.9199,10,No hotel found
128,Bakhtiyarpur,IN,25.4667,85.5167,31,No hotel found
170,Mangrol,IN,21.1167,70.1167,35,No hotel found
279,Bandarbeyla,SO,9.4942,50.8122,60,No hotel found
316,Gao,ML,16.6362,1.6370,8,No hotel found
360,Hobyo,SO,5.3505,48.5268,58,Iftin Hotel Hobyo
426,Butembo,CD,0.1500,29.2833,24,Canada House
436,Puerto Gaitan,CO,4.3133,-72.0816,47,No hotel found
500,Mirpur Sakro,PK,24.5474,67.6277,40,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1400,
    frame_height = 1000,
    size = "Humidity",
    color = "Hotel Name",
    scale = 2.0,
   hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
display(hotel_map)

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)